**SIN Algorithm**

Intuition: Based in d-seperation. No causal relationship between X and Y, if there exists a set of variables wherein X and Y are indpendent when conditined upon.  This is main idea between PIC algorithms. 


A key fact is that the partial correlation can be computed in terms of the inverse of the covariance matrix Σ−1, known as the concentration matrix


In [2]:
import numpy as np
import pandas as pd
import networkx as nx
from itertools import permutations, combinations
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.regression.linear_model import OLS

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

import graphviz
from collections import defaultdict
from scipy.stats import f_oneway

import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams["figure.figsize"] = [30,10]

In [3]:
data = pd.read_csv('output.csv')

In [4]:
# stupid method
# P(A|B) = P(A); then independent
# P(A|B) != P(A); then dependent 

# plt.hist(data['x0'].values)
# plt.scatter(data['x0'].values, data['x1'].values)

'''
The way I think to do this, which was not described in the paper, is to first do a regression of x->y to test for independence
then if there is a correlation, add other variables to see if it remains. 
'''

'\nThe way I think to do this, which was not described in the paper, is to first do a regression of x->y to test for independence\nthen if there is a correlation, add other variables to see if it remains. \n'

In [5]:
def transform(data, target, predictors, max_lag):
    new = data[[target] + predictors]
    ts = new.to_numpy()
    transformed = []

    max_lag = 5
    for i in range(ts.shape[0]):
        if i >= max_lag:
            start = i - max_lag 
            target = ts[:,0][i]
            regressors = []
            for ii in range(ts.shape[1]):
                regressors.append(list(ts[:,ii][start:i]))
            transformed.append([target] + list(np.array(regressors).flatten()))

    transformed = np.array(transformed)

    y = transformed[:,0]
    x = transformed[:,1:]
    
    return x, y 

In [6]:
r1 = OLS(data['x0'].values, data['x2'].values).fit()
r2 = OLS(data['x0'].values, data[['x1', 'x2', 'x3', 'x4']].values).fit()

In [7]:
print(r1.params)
print(r2.params)

[0.45002303]
[ 0.02551748  0.17393629  0.70634603 -0.08420259]


In [8]:
print(r1.pvalues[0])
print(r2.pvalues[0])

4.693864804005188e-17
0.5446394444624337


In [9]:
# this method appears to work, let's check it against R SINFUL Package 

In [10]:
fully_connected_results = list(permutations(data.columns,2))
num_nodes = len(data.columns)
nodes = list(data.columns)
max_lag = 5 

dag = np.eye(num_nodes,num_nodes)

for edge in fully_connected_results:
    y = edge[1]
    x = edge[0]
    sample_nodes = nodes.copy()
    sample_nodes.remove(y)
    sample_nodes.remove(x)
    
    r1 = OLS(data[y].values, data[x].values).fit()
    r2 = OLS(data[y].values, data[sample_nodes].values).fit()

    if r1.pvalues[0] < .05 and r2.pvalues[0] < .05:
        dag[nodes.index(y)][nodes.index(x)] = 1 
        print(x, y)

x2 x3
x3 x2


In [11]:
true = np.array(pd.read_csv("true_graph.csv"))
true

array([[1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]])

In [12]:
print("Precision: " , precision_score(true.flatten(), dag.flatten()))
print("Recal: " , recall_score(true.flatten(), dag.flatten()))
print("F1: " , f1_score(true.flatten(), dag.flatten()))

Precision:  0.8571428571428571
Recal:  0.75
F1:  0.7999999999999999


In [16]:
covaraince_matrix = np.cov(np.array(data).T)
covaraince_matrix

array([[ 2.72496872, -0.03033561,  0.33327629,  1.71371186, -0.07422836],
       [-0.03033561,  1.02747152, -0.0134769 , -0.08111958, -0.06101146],
       [ 0.33327629, -0.0134769 ,  1.26942155,  0.2711533 ,  0.0052886 ],
       [ 1.71371186, -0.08111958,  0.2711533 ,  2.85888705,  0.00607256],
       [-0.07422836, -0.06101146,  0.0052886 ,  0.00607256,  0.98319089]])

In [15]:
np.linalg.inv(covaraince_matrix)

array([[ 0.59959039, -0.00835192, -0.08253121, -0.35192426,  0.0473668 ],
       [-0.00835192,  0.9791831 ,  0.00547264,  0.03214398,  0.05990424],
       [-0.08253121,  0.00547264,  0.81544374, -0.02769267, -0.01010654],
       [-0.35192426,  0.03214398, -0.02769267,  0.56433945, -0.0279113 ],
       [ 0.0473668 ,  0.05990424, -0.01010654, -0.0279113 ,  1.02461664]])